# BUSINESS PROBLEM STAR JEAN'S

Eduardo and Marcelo are two Brazilians, friends and business partners. After several successful business, they are planning to enter the fashion market. US as an E-commerce business model.

The initial idea is to enter the market with just one product and for a specific audience, in this case the product would be Jeans for the male audience. The objective is to maintain the operating cost low and scale as they get customers.

However, even with the input product and audience defined, the two partners do not have experience in this fashion market and therefore they don't know how to define basic things like price, the type of pants and the material for the manufacture of each piece.

Thus, the two partners hired a Data Science consultancy to answer the following questions:

* What is the best selling price for the pants?
* How many types of pants and yours colors for initial product?
* What are the raw materials needed to make the pants?

The main competitors of the Start Jeans company are the American companies H&M and Macys.

# 1. Imports

In [1]:
# webscrapping
import requests
from bs4 import BeautifulSoup

# date
from datetime import datetime

# data manipulation
import pandas as pd
import numpy as np

# monitoring
from tqdm import tqdm

# regex
import re

# database
import sqlite3
from sqlalchemy import create_engine

# 2. WebScrapping (EXTRACT)

In [2]:
# extract data from competitors
url = 'https://www2.hm.com/en_us/men/products/jeans.html'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36"}

# request
page = requests.get(url, headers = headers)

In [3]:
# instantiate BeautifulSoup
soup = BeautifulSoup(page.text, 'html.parser')

# find total number of products
total_item = soup.find_all('h2', class_ = 'load-more-heading')[0].get('data-total')

# find number of items shown in page
items_page = soup.find_all('h2', class_ = 'load-more-heading')[0].get('data-items-shown')

# get number of pages
pages = np.ceil(int(total_item) / int(items_page))

# get section with all products
products = soup.find('ul', class_ = 'products-listing small')

# get list of products overall
product_list = soup.find_all('article', class_ = 'hm-product-item')

len(product_list)

36

## 2.1. Generate table

Table format: 

Id | Category | Name | Price | Color | Decomposition

### 2.1.1 Id, Category, Name, Price

In [4]:
# product_id
product_id = [p.get('data-articlecode') for p in product_list]

# product_category
product_category = [p.get('data-category') for p in product_list]

In [5]:
# get list of products to get name
product_list = products.find_all('a', class_ = 'link')

# product_name
product_name = [p.get_text() for p in product_list]

In [6]:
# get list of products to get price
product_list = products.find_all('span', class_ = 'price regular')

# product_price
product_price = [p.get_text() for p in product_list]

In [7]:
# pass data to DataFrame
df_products = pd.DataFrame([product_id, product_category, product_name, product_price]).T
df_products.columns = ['product_id', 'product_category', 'product_name', 'product_price']

df_products.head()

,product_id,product_category,product_name,product_price
0,0985197005,men_jeans_slim,Slim Jeans,$ 19.99
1,0690449051,men_jeans_ripped,Skinny Jeans,$ 39.99
2,1004476004,men_jeans_slim,Freefit® Slim Jeans,$ 49.99
3,0811993028,men_jeans_regular,Regular Jeans,$ 29.99
4,1013317004,men_jeans_regular,Hybrid Regular Tapered Joggers,$ 39.99


### 2.1.2. Color, Decomposition

In [8]:
# getting color for all products
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36"}
cols = ['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size', 'More sustainable materials']
df_pattern = pd.DataFrame(columns = cols)
df_compositions = pd.DataFrame()
aux = []

for index in tqdm(product_id):
    url = f'https://www2.hm.com/en_us/productpage.{index}.html'
    
    # request
    page = requests.get(url, headers = headers)
    
    # instantiate BeatifulSoup
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # COLOR
    ## product list
    product_list = soup.find_all('a', class_ = 'filter-option miniature active') + soup.find_all('a', class_ = 'filter-option miniature')
    
    ## color name
    color_name = [p.get('data-color') for p in product_list]
    
    # ID FOR MERGE
    ## product_id
    color_product_id = [p.get('data-articlecode') for p in product_list]
    
    # pass to dataframe
    df_color = pd.DataFrame([color_product_id, color_name]).T
    df_color.columns = ['product_id', 'color_name']
    
    for color_index in df_color['product_id']:
        url = f'https://www2.hm.com/en_us/productpage.{color_index}.html'
    
        # request
        page = requests.get(url, headers = headers)

        # instantiate BeatifulSoup
        soup = BeautifulSoup(page.text, 'html.parser')
        
        # product name
        product_name = soup.find_all('h1', class_ = 'primary product-item-headline')
        product_name = product_name[0].get_text()
        
        # product price
        product_price = soup.find_all('div', class_ = 'primary-row product-item-price')
        product_price = re.findall(r'\d+\.?\d+', product_price[0].get_text())[0]
        
        # COMPOSITION
        ## composition list
        product_composition_list = soup.find_all('div', class_ = 'pdp-description-list-item')

        ## composition names
        product_composition = [list(filter(None, c.get_text().split('\n'))) for c in product_composition_list]

        # pass to dataframe
        df_composition = pd.DataFrame(product_composition).T

        # set columns
        df_composition.columns = df_composition.iloc[0]

        # delete first row and fill na
        df_composition = df_composition.iloc[1:].fillna(method = 'ffill')

        # remove pocket lining, shell and lining
        df_composition['Composition'] = df_composition['Composition'].str.replace('Pocket lining: ', '', regex = True)
        df_composition['Composition'] = df_composition['Composition'].str.replace('Shell: ', '', regex = True)
        df_composition['Composition'] = df_composition['Composition'].str.replace('Lining: ', '', regex = True)

        # garantee same number of columns
        df_composition = pd.concat([df_pattern, df_composition], axis = 0).reset_index(drop = True)

        # rename columns
        df_composition.columns = ['product_id', 'composition', 'fit', 'product_safety', 'size', 'more_sustainable_materials']
        
        # set product name and price
        df_composition['product_name'] = product_name
        df_composition['product_price'] = product_price   
        
        # keep track on new columns
        aux = aux + df_composition.columns.to_list()

        # merge color and composition
        df_composition = df_composition.merge(df_color, how = 'left', on = 'product_id')

        # all details from products
        df_compositions = pd.concat([df_compositions, df_composition], axis = 0).reset_index(drop = True)

# generate style id + color id
df_compositions['style_id'] = df_compositions['product_id'].apply(lambda x: x[:-3])
df_compositions['color_id'] = df_compositions['product_id'].apply(lambda x: x[-3:])

# scrapy datetime
df_compositions['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

df_compositions.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [02:36<00:00,  4.34s/it]


,product_id,composition,fit,product_safety,size,more_sustainable_materials,product_name,product_price,color_name,style_id,color_id,scrapy_datetime
0,0985197005,"Cotton 99%, Elastane 1%",Slim fit,NaN,NaN,Recycled cotton 20%,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Dark denim blue,0985197,005,2021-10-14 00:12:26
1,0985197005,"Polyester 65%, Cotton 35%",Slim fit,NaN,NaN,Recycled cotton 20%,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Dark denim blue,0985197,005,2021-10-14 00:12:26
2,0985197001,Cotton 100%,Slim fit,NaN,"The model is 189cm/6'2"" and wears a size 32/32",NaN,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Black,0985197,001,2021-10-14 00:12:26
3,0985197001,"Cotton 98%, Elastane 2%",Slim fit,NaN,"The model is 189cm/6'2"" and wears a size 32/32",NaN,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Black,0985197,001,2021-10-14 00:12:26
4,0985197002,"Cotton 99%, Elastane 1%",Slim fit,NaN,"The model is 182cm/6'0"" and wears a size 31/32",Recycled cotton 20%,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Midnight blue,0985197,002,2021-10-14 00:12:26


### 2.1.3. Merge with main product list

In [9]:
# final merge
data_raw = df_compositions.copy()
data_raw.head()

,product_id,composition,fit,product_safety,size,more_sustainable_materials,product_name,product_price,color_name,style_id,color_id,scrapy_datetime
0,0985197005,"Cotton 99%, Elastane 1%",Slim fit,NaN,NaN,Recycled cotton 20%,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Dark denim blue,0985197,005,2021-10-14 00:12:26
1,0985197005,"Polyester 65%, Cotton 35%",Slim fit,NaN,NaN,Recycled cotton 20%,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Dark denim blue,0985197,005,2021-10-14 00:12:26
2,0985197001,Cotton 100%,Slim fit,NaN,"The model is 189cm/6'2"" and wears a size 32/32",NaN,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Black,0985197,001,2021-10-14 00:12:26
3,0985197001,"Cotton 98%, Elastane 2%",Slim fit,NaN,"The model is 189cm/6'2"" and wears a size 32/32",NaN,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Black,0985197,001,2021-10-14 00:12:26
4,0985197002,"Cotton 99%, Elastane 1%",Slim fit,NaN,"The model is 182cm/6'0"" and wears a size 31/32",Recycled cotton 20%,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Midnight blue,0985197,002,2021-10-14 00:12:26


In [10]:
data_raw.shape

(423, 12)

In [11]:
# lower column names
data_raw.columns = data_raw.columns.str.lower()

In [12]:
data_raw.to_csv('./data/data_raw.csv', index = False)

# 3. Data Cleaning (TRANSFORMING)

In [13]:
df = pd.read_csv('./data/data_raw.csv')
df.head()

,product_id,composition,fit,product_safety,size,more_sustainable_materials,product_name,product_price,color_name,style_id,color_id,scrapy_datetime
0,985197005,"Cotton 99%, Elastane 1%",Slim fit,NaN,NaN,Recycled cotton 20%,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Dark denim blue,985197,5,2021-10-14 00:12:26
1,985197005,"Polyester 65%, Cotton 35%",Slim fit,NaN,NaN,Recycled cotton 20%,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Dark denim blue,985197,5,2021-10-14 00:12:26
2,985197001,Cotton 100%,Slim fit,NaN,"The model is 189cm/6'2"" and wears a size 32/32",NaN,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Black,985197,1,2021-10-14 00:12:26
3,985197001,"Cotton 98%, Elastane 2%",Slim fit,NaN,"The model is 189cm/6'2"" and wears a size 32/32",NaN,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Black,985197,1,2021-10-14 00:12:26
4,985197002,"Cotton 99%, Elastane 1%",Slim fit,NaN,"The model is 182cm/6'0"" and wears a size 31/32",Recycled cotton 20%,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Midnight blue,985197,2,2021-10-14 00:12:26


In [14]:
# check missing values
df.isna().sum()

product_id                      0
composition                     0
fit                             0
product_safety                395
size                          246
more_sustainable_materials    201
product_name                    0
product_price                   0
color_name                      0
style_id                        0
color_id                        0
scrapy_datetime                 0
dtype: int64

In [15]:
# check data types
df.dtypes

product_id                      int64
composition                    object
fit                            object
product_safety                 object
size                           object
more_sustainable_materials     object
product_name                   object
product_price                 float64
color_name                     object
style_id                        int64
color_id                        int64
scrapy_datetime                object
dtype: object

In [35]:
len(df['product_id'].unique())

121

In [51]:
# product_id
df_data = df_compositions.dropna(subset = ['product_id'])

# product name - change format and remove tabs and newlines
df_data['product_name'] = df_data['product_name'].str.replace('\n', '')
df_data['product_name'] = df_data['product_name'].str.replace('\t', '')
df_data['product_name'] = df_data['product_name'].str.replace('  ', '')
df_data['product_name'] = df_data['product_name'].str.replace(' ', '_').str.lower()

# product price - remove $
df_data['product_price'] = df_data['product_price'].astype(float)

# # scrapy datetime
# df_data['scrapy_datetime'] = pd.to_datetime(df_data['scrapy_datetime'], errors = 'coerce')

# # style id
# df_data['style_id'] = df_data['style_id'].astype(int)

# # color id
# df_data['color_id'] = df_data['color_id'].astype(int)

# color name - change format
df_data['color_name'] = df_data['color_name'].str.replace(' ', '_').str.lower()\

# fit
df_data['fit'] = df_data['fit'].apply(lambda x: x.lower().replace(' ', '_') if pd.notnull(x) else x)

# size number
df_data['size_number'] = df_data['size'].apply(lambda x: re.search('\d{3}cm', x).group(0) if pd.notnull(x) else x)
df_data['size_number'] = df_data['size_number'].apply(lambda x: re.search('\d+', x).group(0) if pd.notnull(x) else x)

# size model
df_data['size_model'] = df_data['size'].str.extract('(\d+/\\d+)')

# # composition
# df_data = df_data[~df_data['composition'].str.contains('Pocket lining:', na = False)]
# df_data = df_data[~df_data['composition'].str.contains('Lining:', na = False)]
# df_data = df_data[~df_data['composition'].str.contains('Shell:', na = False)]
# df_data = df_data[~df_data['composition'].str.contains('Pocket:', na = False)]

# # drop duplicates
# df_data = df_data.drop_duplicates(subset = ['product_id', 'product_category', 'product_name', 'product_price',
#                                   'scrapy_datetime', 'style_id', 'color_id', 'color_name', 'fit'],
#                         keep = 'last')

# # reset index
# df_data = df_data.reset_index(drop = True)

# break composition by comma
df1 = df_data['composition'].str.split(',', expand = True).reset_index(drop = True)

df_data.head(2)

,product_id,composition,fit,product_safety,size,more_sustainable_materials,product_name,product_price,color_name,style_id,color_id,scrapy_datetime,size_number,size_model
0,0985197005,"Cotton 99%, Elastane 1%",slim_fit,NaN,NaN,Recycled cotton 20%,slim_jeans,19.99,dark_denim_blue,0985197,005,2021-10-14 00:12:26,NaN,NaN
1,0985197005,"Polyester 65%, Cotton 35%",slim_fit,NaN,NaN,Recycled cotton 20%,slim_jeans,19.99,dark_denim_blue,0985197,005,2021-10-14 00:12:26,NaN,NaN


In [52]:
# cotton / polyester / elastane / elasterell
df_ref = pd.DataFrame(index = np.arange(len(df_data)), columns = ['cotton', 'polyester', 'elastane', 'elasterell'])

# cotton
df_cotton_0 = df1.loc[df1[0].str.contains('Cotton', na = True), 0]
df_cotton_0.name = 'cotton'

df_cotton_1 = df1.loc[df1[1].str.contains('Cotton', na = True), 1]
df_cotton_1.name = 'cotton'

## combine cotton
df_cotton = df_cotton_0.combine_first(df_cotton_1)

df_ref = pd.concat([df_ref, df_cotton], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep = 'last')]

# polyester
df_polyester_0 = df1.loc[df1[0].str.contains('Polyester', na = True), 0]
df_polyester_0.name = 'polyester'

df_polyester_1 = df1.loc[df1[1].str.contains('Polyester', na = True), 1]
df_polyester_1.name = 'polyester'

## combine polyester
df_polyester = df_polyester_0.combine_first(df_polyester_1)

df_ref = pd.concat([df_ref, df_polyester], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep = 'last')]

# elastane
df_elastane_1 = df1.loc[df1[1].str.contains('Elastane', na = True), 1]
df_elastane_1.name = 'elastane'

df_elastane_2 = df1.loc[df1[2].str.contains('Elastane', na = True), 2]
df_elastane_2.name = 'elastane'

df_elastane_3 = df1.loc[df1[3].str.contains('Elastane', na = True), 3]
df_elastane_3.name = 'elastane'

## combine elastane
df_elastane_c2 = df_elastane_1.combine_first(df_elastane_2)
df_elastane = df_elastane_c2.combine_first(df_elastane_3)

df_ref = pd.concat([df_ref, df_elastane], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep = 'last')]

# elasterell
df_elasterell = df1.loc[df1[1].str.contains('Elasterell', na = True), 1]
df_elasterell.name = 'elasterell'

df_ref = pd.concat([df_ref, df_elasterell], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep = 'last')]

# join combine df with product_id
df_aux = pd.concat([df_data['product_id'].reset_index(drop = True), df_ref], axis = 1)

# format composition data
df_aux['cotton'] = df_aux['cotton'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)
df_aux['polyester'] = df_aux['polyester'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)
df_aux['elastane'] = df_aux['elastane'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)
df_aux['elasterell'] = df_aux['elasterell'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)

# final join
df_aux = df_aux.groupby('product_id').max().reset_index().fillna(0)

df_data = pd.merge(df_data, df_aux, on = 'product_id', how = 'left')

# drop columns
df_data = df_data.drop(columns = ['size', 'product_safety', 'composition'])

# drop duplicates
df_data = df_data.drop_duplicates(subset = ['product_id'], keep = 'last').reset_index(drop = True)
df_data.shape

(121, 15)

In [60]:
df_data.head()

,product_id,fit,more_sustainable_materials,product_name,product_price,color_name,style_id,color_id,scrapy_datetime,size_number,size_model,cotton,polyester,elastane,elasterell
44,0811993028,regular_fit,Recycled cotton 20%,regular_jeans,29.99,denim_blue,0811993,028,2021-10-14 00:12:26,185,31/32,0.99,0.0,0.01,0.0
46,0811993001,regular_fit,NaN,regular_jeans,12.99,black/washed_out,0811993,001,2021-10-14 00:12:26,NaN,NaN,0.98,0.0,0.02,0.0
48,0811993002,regular_fit,NaN,regular_jeans,29.99,denim_blue,0811993,002,2021-10-14 00:12:26,NaN,NaN,0.98,0.0,0.02,0.0
50,0811993003,regular_fit,Recycled cotton 20%,regular_jeans,29.99,dark_denim_blue,0811993,003,2021-10-14 00:12:26,NaN,NaN,0.98,0.0,0.02,0.0
52,0811993006,regular_fit,NaN,regular_jeans,14.99,graphite_gray,0811993,006,2021-10-14 00:12:26,NaN,NaN,0.98,0.0,0.02,0.0


In [53]:
df.to_csv('./data/data_clean.csv')

# 4. Save data in a database (LOAD)

In [54]:
df_clean = pd.read_csv('./data/data_clean.csv')

In [66]:
query_showroom_schema = """
CREATE TABLE showroom (
    product_id INTEGER, 
    product_category TEXT, 
    product_name TEXT, 
    product_price REAL,
    scrapy_datetime TEXT, 
    style_id INTEGER, 
    color_id TEXT, 
    color_name TEXT, 
    fit TEXT,
    more_sustainable_materials TEXT,
    size_number REAL, 
    size_model TEXT, 
    cotton REAL,
    polyester REAL,
    elastane REAL,
    elasterell REAL
    )
"""

In [67]:
# create engine sqlalchemy
# conn = create_engine('sqlite:///./database/hm_db.sqlite', echo = False)

In [69]:
# connect to database
conn = sqlite3.connect('./database/hm_db.sqlite')
# cursor = conn.execute(query_showroom_schema)
# conn.commit()
# conn.close()

In [70]:
# insert data to table
df_data.to_sql('showroom', con = conn, if_exists = 'append', index = False)

In [72]:
# extract data from database
query = """
SELECT
    *
FROM
    showroom
"""
df = pd.read_sql_query(query, conn)
df.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,fit,more_sustainable_materials,size_number,size_model,cotton,polyester,elastane,elasterell
0,811993028,None,regular_jeans,29.99,2021-10-14 00:12:26,811993,028,denim_blue,regular_fit,Recycled cotton 20%,185.0,31/32,0.99,0.0,0.01,0.0
1,811993001,None,regular_jeans,12.99,2021-10-14 00:12:26,811993,001,black/washed_out,regular_fit,None,NaN,None,0.98,0.0,0.02,0.0
2,811993002,None,regular_jeans,29.99,2021-10-14 00:12:26,811993,002,denim_blue,regular_fit,None,NaN,None,0.98,0.0,0.02,0.0
3,811993003,None,regular_jeans,29.99,2021-10-14 00:12:26,811993,003,dark_denim_blue,regular_fit,Recycled cotton 20%,NaN,None,0.98,0.0,0.02,0.0
4,811993006,None,regular_jeans,14.99,2021-10-14 00:12:26,811993,006,graphite_gray,regular_fit,None,NaN,None,0.98,0.0,0.02,0.0


In [ ]:
# # command - UPDATE
# query = """
# UPDATE showroom
# SET product_category = 'tshirt'
# WHERE product_id = 690449051
# """

#command - DROP
# query = """
# DROP TABLE showroom
# """

# command - ALTER TABLE
# query = """
# ALTER TABLE showroom
# RENAME showroom_two
# """

# command - CREATE INDEX
query = """
CREATE INDEX idx_product_id
ON showroom (product_id)
"""

conn = sqlite3.connect('./database/hm_db.sqlite')
cursor = conn.execute(query)
conn.commit()

In [ ]:
conn.close()